In [1]:
import time
import json
import requests
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
import os

In [2]:
from datetime import datetime

In [3]:
from pandas.io import sql
from sqlalchemy import create_engine

In [4]:
import zipfile, urllib.request, shutil

In [5]:
path_notebook = os.getcwd()

In [6]:
#Inicio da execução do notebook
start_general = datetime.now()

## Definir funções

In [7]:
def cadastrar_fonte_dados (nome_site, site_fonte_dados, nome_fonte_dados):
    
    fonte_dados = [[nome_site, site_fonte_dados, nome_fonte_dados]]

    colunas = ['Nome_Site', 'Site_Fonte_Dados', 'Nome_Fonte_Dados']

    fonte_dados = pd.DataFrame(fonte_dados, columns = colunas)

    with engine.connect() as conn, conn.begin():
        fonte_dados.to_sql('stg_Fonte_Dados', conn, if_exists='replace', index=False)


    ## Executar procedure versionamento

    conexao = engine.connect()

    consulta_sql = "CALL Stage.Versiona_stg_Fonte_Dados_Internet();"

    with engine.begin() as conn:
        conn.execute(consulta_sql)


    ## Deletar dados tabela stage

    consulta_sql = """
        DELETE FROM Stage.stg_Fonte_Dados
        WHERE TRUE;
    """

    with engine.begin() as conn:
        conn.execute(consulta_sql)

    conexao.close()

    ### Identificar ID Fonte de dados

    conn = engine.connect()

    consulta_sql = """
        SELECT id_Fonte_Dados
        FROM Internet_db.Fonte_Dados_Internet
        WHERE Site_Fonte_Dados = '""" +  site_fonte_dados  + """'

    """

    with engine.begin() as conn:
        result_query = conn.execute(consulta_sql).fetchall()

    return result_query[0][0]

In [8]:
def consultar_fonte_dados(nome_site, site_fonte_dados, nome_fonte_dados):
    
    conn = engine.connect()

    consulta_sql = """
        SELECT id_Fonte_Dados
        FROM Internet_db.Fonte_Dados_Internet
        WHERE Site_Fonte_Dados = '""" +  site_fonte_dados  + """'

    """

    with engine.begin() as conn:
        result_query = conn.execute(consulta_sql).fetchall()


    if len(result_query) != 0:
        return result_query[0][0]

    else:
        print('Fonte de dados não está previamente cadastrada')
        print('Fonte de dados será cadastrada agora')

        return cadastrar_fonte_dados (nome_site, site_fonte_dados, nome_fonte_dados)

In [9]:
def inserir_dados_stg(nome_data_frame, nome_banco_tabela, nome_tabela, nome_banco_procedure, nome_procedure):

    # Inserir dados tabela
    with engine.connect() as conn, conn.begin():
        nome_data_frame.to_sql(nome_tabela, conn, if_exists='replace', index=False)



    ## Executar procedure versionamento
    conexao = engine.connect()

    consulta_sql = "CALL "  + nome_banco_procedure + "." + nome_procedure + "();"

    with engine.begin() as conn:
        conn.execute(consulta_sql)



    ## Deletar dados tabela

    consulta_sql = """
        DELETE FROM """ + nome_banco_tabela + """.""" + nome_tabela + """
        WHERE TRUE;
    """

    with engine.begin() as conn:
        conn.execute(consulta_sql)

    conexao.close()

In [10]:
def importar_arquivo_zip(url_name, local_file_name):
    with urllib.request.urlopen(url_name) as response, open(local_file_name, 'wb') as out_file:
        shutil.copyfileobj(response, out_file)
        with zipfile.ZipFile(local_file_name) as zf:
            zf.extractall()

In [11]:
def renomear_colunas(nome_df, nomes_colunas):
    for i in range(0, len(nomes_colunas), 1):
        #print (nomes_colunas[i])
        nome_df.rename(columns = {
            nome_df.columns[i] : nomes_colunas[i]
        }, inplace = True) 

## Criar conexão banco dados

In [12]:
#texto = open('/root/airflow/scripts/config/conexao_db.txt')
texto = open('conexao_db.txt')
conexao_db = texto.read()
texto.close()

In [13]:
engine = create_engine(conexao_db, encoding='utf-8')

In [14]:
print(engine.table_names())

['stg_Capital_Country', 'stg_Code_Country', 'stg_Country', 'stg_DDD_Municipio', 'stg_Distrito_IBGE', 'stg_Estado_IBGE', 'stg_Fonte_Dados', 'stg_Mesoregiao_IBGE', 'stg_Microregicao_IBGE', 'stg_Municipio_IBGE', 'stg_Pais_IBGE', 'stg_Population_Country', 'stg_Regiao_IBGE', 'stg_Regiao_Imediata_IBGE', 'stg_Regiao_Intermediaria_IBGE', 'stg_Regiao_Mundo_IBGE', 'stg_Sub_Distrito_IBGE', 'stg_Subregiao_Mundo_IBGE']


### Datas de vigências

In [15]:
data_inicio_vigencia = datetime.now().strftime('%Y-%m-%d')

In [16]:
data_fim_vigencia = '9999-12-31'

# Importar base de dados de países

In [17]:
url_download = r'https://ftp.ibge.gov.br/Pib_Municipios/2018/base/base_de_dados_2010_2018_xls.zip'
nome_arquivo_local = 'pib_2010_2018.zip'
nome_arquivo_original = 'PIB dos Municípios - base de dados 2010-2018.xls'

In [18]:
importar_arquivo_zip(url_download, nome_arquivo_local)
data_extracao = datetime.now()

In [ ]:
archive = zipfile.ZipFile(nome_arquivo_local, 'r')

In [104]:
local_zip_file = archive.open(nome_arquivo_original)

In [105]:
pib = pd.read_excel(local_zip_file)

### Identificar ID Fonte de dados

In [106]:
nome_site = 'IBGE'
site_fonte_dados = url_download
nome_fonte_dados = 'Produto Interno Bruto dos Municípios - 2018'

In [107]:
id_fonte_dados = consultar_fonte_dados(nome_site, site_fonte_dados, nome_fonte_dados)

In [108]:
print('id_fonte_dados:' , id_fonte_dados)

id_fonte_dados: 21


## Conversões

In [109]:
nomes_colunas = [
    'ano_PIB'
    ,'id_Regiao'
    ,'nome_Regiao'
    ,'id_Estado'
    ,'sigla_Estado'
    ,'nome_Estado'
    ,'id_Municipio'
    ,'nome_Municipio'
    ,'nome_Regiao_Metropolitana'
    ,'id_Mesoregiao'
    ,'nome_Mesoregiao'
    ,'id_Microregiao'
    ,'nome_Microregiao'
    ,'id_Regiao_Imediata'
    ,'nome_Regiao_Imediata'
    ,'municipio_Regiao_Imediata'
    ,'id_Regiao_Intermediaria'
    ,'nome_Regiao_Intermediaria'
    ,'municipio_Regiao_Intermediaria'
    ,'codigo_Concentracao_Urbana'
    ,'nome_Concentracao_Urbana'
    ,'tipo_Concentracao_Urbana'
    ,'codigo_Arranjo_Populacional'
    ,'nome_Arranjo_Populacional'
    ,'hierarquia_Urbana'
    ,'hierarquia_Urbana_Principais'
    ,'codigo_Regiao_Rural'
    ,'nome_Regiao_Rural'
    ,'nome_Regiao_Rural_Nucleo'
    ,'amazonia_Legal'
    ,'semiarido'
    ,'regiao_Sao_Paulo'
    ,'valor_Bruto_Agropecuaria'
    ,'valor_Bruto_Industria'
    ,'valor_Bruto_Servicos'
    ,'valor_Bruto_Administracao_Publica'
    ,'valor_Bruto_Total'
    ,'valor_Impostos_Liquido'
    ,'valor_PIB'
    ,'valor_PIB_percapita'
    ,'atividade_Primeiro_Maior_Valor'
    ,'atividade_Segundo_Maior_Valor'
    ,'atividade_Terceiro_Maior_Valor'
]

In [110]:
renomear_colunas(pib, nomes_colunas)

#### Ajustar colunas valore por 1000

In [111]:
pib['valor_Bruto_Agropecuaria'] = pib['valor_Bruto_Agropecuaria'] *1000

In [112]:
pib['valor_Bruto_Industria'] = pib['valor_Bruto_Industria'] *1000

In [113]:
pib['valor_Bruto_Servicos'] = pib['valor_Bruto_Servicos'] *1000

In [114]:
pib['valor_Bruto_Administracao_Publica'] = pib['valor_Bruto_Administracao_Publica'] *1000

In [115]:
pib['valor_Bruto_Total'] = pib['valor_Bruto_Total'] *1000

In [116]:
pib['valor_Impostos_Liquido'] = pib['valor_Impostos_Liquido'] *1000

In [117]:
pib['valor_PIB'] = pib['valor_PIB'] *1000

In [118]:
pib['populacao_Projetada'] = pib['valor_PIB'] / pib['valor_PIB_percapita'] 

In [119]:
pib['populacao_Projetada'] = pib['populacao_Projetada'].round(0)

In [120]:
pib['tipo_Valor'] = 'Preço corrente do ano referência'

In [134]:
pib['data_Extracao'] = data_extracao

In [122]:
pib['ano_Referencia'] = 2018

In [123]:
pib['id_Fonte_Dados'] = id_fonte_dados

In [135]:
pib['data_Inicio_Vigencia'] = data_inicio_vigencia

In [136]:
pib['data_Fim_Vigencia'] = data_fim_vigencia

In [133]:
pib[(pib['sigla_Estado'] == 'SE') & (pib['ano_PIB'] == 2010)].iloc[:, -18: -6].head(3)

,valor_Bruto_Agropecuaria,valor_Bruto_Industria,valor_Bruto_Servicos,valor_Bruto_Administracao_Publica,valor_Bruto_Total,valor_Impostos_Liquido,valor_PIB,valor_PIB_percapita,atividade_Primeiro_Maior_Valor,atividade_Segundo_Maior_Valor,atividade_Terceiro_Maior_Valor,populacao_Projetada
1751,879428.0,1.120482e+06,2.509983e+06,1.019363e+07,1.470352e+07,5.497330e+05,1.525326e+07,6704.73,"Administração, defesa, educação e saúde públic...",Demais serviços,"Eletricidade e gás, água, esgoto, atividades d...",2275.0
1752,23759237.0,7.195914e+06,2.850722e+07,5.961338e+07,1.190757e+08,5.356383e+06,1.244321e+08,6201.14,"Administração, defesa, educação e saúde públic...",Demais serviços,"Agricultura, inclusive apoio à agricultura e a...",20066.0
1753,3547706.0,1.662170e+09,4.975889e+09,1.702071e+09,8.343678e+09,1.260267e+09,9.603945e+09,16821.37,Demais serviços,"Administração, defesa, educação e saúde públic...",Comércio e reparação de veículos automotores e...,570937.0


### Selecionar colunas especificas

In [138]:
nomes_colunas = [
    'id_Fonte_Dados'
    ,'id_Regiao'
    ,'id_Estado'
    ,'id_Municipio'
    ,'id_Mesoregiao'
    ,'id_Microregiao'
    ,'id_Regiao_Imediata'
    ,'id_Regiao_Intermediaria'
    ,'ano_Referencia'
    ,'ano_PIB'
    ,'valor_Bruto_Agropecuaria'
    ,'valor_Bruto_Industria'
    ,'valor_Bruto_Servicos'
    ,'valor_Bruto_Administracao_Publica'
    ,'valor_Bruto_Total'
    ,'valor_Impostos_Liquido'
    ,'valor_PIB'
    ,'valor_PIB_percapita'
    ,'populacao_Projetada'
    ,'tipo_Valor'
    ,'atividade_Primeiro_Maior_Valor'
    ,'atividade_Segundo_Maior_Valor'
    ,'atividade_Terceiro_Maior_Valor'
    ,'data_Extracao'
    ,'data_Inicio_Vigencia'
    ,'data_Fim_Vigencia'
]

In [140]:
pib = pib[nomes_colunas]

In [141]:
pib.head()

,id_Fonte_Dados,id_Regiao,id_Estado,id_Municipio,id_Mesoregiao,id_Microregiao,id_Regiao_Imediata,id_Regiao_Intermediaria,ano_Referencia,ano,...,valor_PIB,valor_PIB_percapita,populacao_Projetada,tipo_Valor,atividade_Primeiro_Maior_Valor,atividade_Segundo_Maior_Valor,atividade_Terceiro_Maior_Valor,data_Extracao,data_Inicio_Vigencia,data_Fim_Vigencia
0,21,1,11,1100015,1102,11006,110005,1102,2018,2010,...,2.620769e+08,10731.18,24422.0,Preço corrente do ano referência,"Administração, defesa, educação e saúde públic...","Pecuária, inclusive apoio à pecuária",Demais serviços,2021-07-14 12:26:49.125082,2021-07-14,9999-12-31
1,21,1,11,1100023,1102,11003,110002,1101,2018,2010,...,1.364694e+09,15103.86,90354.0,Preço corrente do ano referência,"Administração, defesa, educação e saúde públic...",Demais serviços,Comércio e reparação de veículos automotores e...,2021-07-14 12:26:49.125082,2021-07-14,9999-12-31
2,21,1,11,1100031,1102,11008,110006,1102,2018,2010,...,6.961111e+07,11033.62,6309.0,Preço corrente do ano referência,"Administração, defesa, educação e saúde públic...","Pecuária, inclusive apoio à pecuária",Demais serviços,2021-07-14 12:26:49.125082,2021-07-14,9999-12-31
3,21,1,11,1100049,1102,11006,110005,1102,2018,2010,...,1.186494e+09,15095.15,78601.0,Preço corrente do ano referência,"Administração, defesa, educação e saúde públic...",Demais serviços,Comércio e reparação de veículos automotores e...,2021-07-14 12:26:49.125082,2021-07-14,9999-12-31
4,21,1,11,1100056,1102,11008,110006,1102,2018,2010,...,2.220212e+08,13037.06,17030.0,Preço corrente do ano referência,"Administração, defesa, educação e saúde públic...",Demais serviços,Comércio e reparação de veículos automotores e...,2021-07-14 12:26:49.125082,2021-07-14,9999-12-31


### Inserir dados

In [142]:
with engine.connect() as conn, conn.begin():
    pib.to_sql('stg_PIB_Municipio_IBGE', conn, if_exists='replace', index=False)
    

### Gerar informação de Subregião país

In [ ]:
#Fim da execução do notebook
end_general = datetime.now()

In [ ]:
# Write the DataFrame to a BigQuery table

print("Tempo total notebook " + str(end_general - start_general))